In [20]:
from decouple import config
import json
import pymongo
from pprint import pprint
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from bson import json_util

MONGODB_API_KEY = config("MONGODB_API_KEY")
ANNICT_API_KEY = config("ANNICT_API_KEY")
ANNICT_API_URL = "https://api.annict.com/graphql"

# Select your transport with a defined url endpoint
headers = {
    "Content-type": "application/json",
    "Authorization": f"Bearer {ANNICT_API_KEY}"
    }
transport = AIOHTTPTransport(url=ANNICT_API_URL, headers=headers)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True, execute_timeout=300)




In [22]:
# Provide a GraphQL query
query = gql(
    """
query {
  searchWorks(
    orderBy: {field: WATCHERS_COUNT, direction: DESC},
  ) {
    nodes {
      annictId
      title
      titleKana
      watchersCount
      reviewsCount
      reviews {
        nodes {
          id
          ratingOverallState
          ratingMusicState
          ratingStoryState
          ratingAnimationState
          ratingCharacterState
        }
      }
    }
  }
}
"""
)

# res = client.execute(query)
result = await client.execute_async(query)

In [76]:
client = pymongo.MongoClient(MONGODB_API_KEY)
db = client.raw_data
collection_reviews = db.reviews

In [31]:
import pandas as pd


In [77]:
df = pd.json_normalize(result["searchWorks"]["nodes"], ["reviews", "nodes"], ["annictId", "title", "titleKana", "watchersCount", "reviewsCount"])

In [78]:
df.head()

,id,ratingOverallState,ratingMusicState,ratingStoryState,ratingAnimationState,ratingCharacterState,annictId,title,titleKana,watchersCount,reviewsCount
0,UmV2aWV3LTIxNDE=,GREAT,GREAT,GREAT,GREAT,GREAT,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10749,101
1,UmV2aWV3LTIyNTM=,GOOD,GREAT,GOOD,AVERAGE,AVERAGE,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10749,101
2,UmV2aWV3LTI0MDI=,GREAT,GOOD,GREAT,GOOD,GREAT,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10749,101
3,UmV2aWV3LTE1MzI2,GOOD,GREAT,GREAT,BAD,GREAT,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10749,101
4,UmV2aWV3LTQ2Nzc3,GREAT,None,None,None,None,2108,魔法少女まどか☆マギカ,まほうしょうじょまどかまぎか,10749,101


In [79]:
collection_reviews.insert_many(df.to_dict("records"))